# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [2]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [4]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""


biology_template = """你是一位非常有经验的生物学教授。
你擅长通过生动的例子和实验来解释复杂的生物学概念。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""

computer_science_template = """你是一位非常创新的计算机科学教授。
你擅长通过实际编程示例和项目来教授计算机科学的原理和应用。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""

chinese_language_template = """你是一位非常博学的汉语言文学教授。
你擅长通过经典文学作品和历史背景来讲解汉语言文学的精髓。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""

In [5]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物学问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机科学",
        "description": "适用于回答计算机科学问题",
        "prompt_template": computer_science_template,
    },
    {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": chinese_language_template,
    },
]

In [6]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [7]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

D:\weiar2\app\miniconda3\envs\openai-quickstart\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
D:\weiar2\app\miniconda3\envs\openai-quickstart\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(


In [8]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [9]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [10]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [11]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物学问题', '计算机科学: 适用于回答计算机科学问题', '汉语言文学: 适用于回答汉语言文学问题']


In [12]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物学问题
计算机科学: 适用于回答计算机科学问题
汉语言文学: 适用于回答汉语言文学问题


In [13]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [14]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物学问题
计算机科学: 适用于回答计算机科学问题
汉语言文学: 适用于回答汉语言文学问题

<< INPUT >>
{inpu

In [15]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [16]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？'}
> Finished chain.
{'input': '黑体辐射是什么？', 'text': '\n黑体辐射是指处于热平衡状态的物体所发出的电磁辐射，它的频率和强度与温度有关。它的特点是在所有频率上都有辐射，而且其频率和强度的分布仅取决于物体的温度，而与物体的性质无关。这是一个重要的物理现象，在热学、光学和天文学等领域都有应用。'}


In [17]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？'}
> Finished chain.
{'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？', 'text': '\n\n答案：43，因为43是大于40的最小的质数，且44除以3余数为2，加一后能被3整除。'}


In [18]:
router_chain.verbose = True

In [19]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '什么是黑洞？'}
> Finished chain.
{'input': '什么是黑洞？', 'text': '\n\n黑洞是一种密度极高的天体，它的引力非常强大，甚至连光都无法逃离它的吸引力。这种天体是由恒星坍塌形成的，其核心的密度会无限增加，直到无限大，因此它产生的引力也会无限增大。这使得黑洞周围的空间产生了弯曲，形成了一个称为事件视界的区域，任何落入这个区域的物质都无法逃脱。黑洞的存在可以通过它产生的引力影响周围物体的运动来证明。'}


In [20]:
print(chain.invoke("欧拉公式是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
数学: {'input': '什么是欧拉公式？'}
> Finished chain.
{'input': '什么是欧拉公式？', 'text': '\n\n欧拉公式是数学家欧拉提出的一条重要的数学公式，也被称为欧拉方程。它的数学表达式为e^(ix)=cos(x)+isin(x)，其中e为自然对数的底数，i为虚数单位，x为任意实数。这个公式将三种基本的数学常数e、i、π联系在一起，可以被看作是数学中最具有美感的公式之一。它在数学分析、复变函数、微积分等多个领域都有广泛的应用。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [21]:
print(chain.invoke("DNA是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': 'DNA是什么？'}
> Finished chain.
{'input': 'DNA是什么？', 'text': '\n\nDNA是脱氧核糖核酸，它是生物体内存储遗传信息的分子。它由四种碱基（腺嘌呤、鸟嘌呤、胸腺嘧啶和胞嘧啶）组成，并以双螺旋结构呈现。DNA通过遗传信息的传递来控制生物体的生长、发育和功能。它也是生物学研究中非常重要的一部分，因为它可以帮助我们了解生物体的进化和遗传变化。'}


In [22]:
print(chain.invoke("怎么用python实现冒泡排序算法？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机科学: {'input': '怎么用python实现冒泡排序算法？'}
> Finished chain.
{'input': '怎么用python实现冒泡排序算法？', 'text': '\n\n冒泡排序算法是一种简单的排序算法，它的基本思想是通过不断比较相邻的两个元素，将较大的元素向后移动，最终实现整个数组的有序排列。以下是用Python实现冒泡排序算法的代码：\n\n```\n# 定义一个函数bubble_sort，用于实现冒泡排序\ndef bubble_sort(arr):\n    # 获取数组的长度\n    n = len(arr)\n\n    # 遍历数组，进行n-1轮比较\n    for i in range(n - 1):\n        # 每轮比较，都从数组的第一个元素开始，直到倒数第i+1个元素\n        for j in range(n - i - 1):\n            # 如果相邻的两个元素大小顺序不正确，就进行交换\n            if arr[j] > arr[j + 1]:\n                arr[j], arr[j + 1] = arr[j + 1], arr[j]\n\n    # 返回排序'}


In [24]:
print(chain.invoke("红楼梦是谁写的？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
汉语言文学: {'input': '红楼梦是谁写的？'}
> Finished chain.
{'input': '红楼梦是谁写的？', 'text': '\n\n红楼梦是清朝作家曹雪芹所著的长篇小说。'}
